THIS NOTEBOOK REFERS THE RESULT OF TASK 2 OF QUANTIUM VIRTUAL INTERSHIP OF CHIPS ANALYSIS.TRY TO LEARN SOMETHING INSTEAD OF COPYING IT TO COMPLETE TASK.IF YOU NOW SOMETHING BETTER PLZ LET ME KNOW IN COMMENTS AND AS WELL AS OTHERS

THANKS

IN CASE IF YOU DON'T LAND ON MY TASK 1 RESULT HERE IS LINK:https://www.kaggle.com/code/randhirasdataanalyst/quantium-chips-analysis-task1

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries to remove filter error and re
import re
import warnings
warnings.filterwarnings('ignore')
#for visualization
import seaborn as sns
import matplotlib.pyplot as plt

                                                                TASK 2
                                                      QUANTIUM VIRTUAL INTERNSHIP

Task to do:
Define metrics to select control stores.

Analyze trial stores against controls.

Use R/Python for data analysis and visualization and summarise findings and provide recommendations.

THIS IS WHAT WE NEED TO FIND:
                                                    
                                                    total sales revenue
                                                    
                                                    total number of customers
                                                    
                                                    average number of transactions per customer


I know task 2 is confusing and hard to understand as beginner so, let me explain what exactly we need to do in this task:
as we have already given a trial store 77,86 and 88 we have to find out control store from a dataset whose value matches for given task like total sales revenue,total number of customer and average number of transaction per customer in a period and analyze the factor how vary for control store and trial store 

In [ ]:
df=pd.read_csv('/kaggle/input/quantium-task2-dataset/QVI_data.csv')
df.head()

In [ ]:
df.STORE_NBR.unique()

In [ ]:
#let's add new column to dataset with year+month
df['DATE']=pd.to_datetime(df['DATE'])
df['YEARMONTH']=df['DATE'].dt.year*100+df['DATE'].dt.month
df.head()

Finding the metrics for analysis over time :


In [ ]:
metrics=df.groupby(['STORE_NBR','YEARMONTH']).agg(
    TOTAL_SALES=('TOT_SALES',sum),
    TOTAL_CUSTOMER=('LYLTY_CARD_NBR',pd.Series.nunique),
    nTXNperCUSTOMER=('TXN_ID',lambda x: x.nunique()/df['LYLTY_CARD_NBR'].nunique()),
    nCHIPSperTXN=('PROD_QTY',lambda x: x.sum()/df['TXN_ID'].nunique()),
    avgPRICEperUNIT=('TOT_SALES',lambda x: x.sum()/df['PROD_QTY'].sum())
).reset_index()

In [ ]:
metrics

Filtering pre-trial period

In [ ]:
pre_trial_period=metrics[metrics['YEARMONTH']<201902]
pre_trial_period

selecting control store


To ensure a fair comparison, we need to select control stores that match the trial stores in terms of sales trends and
performance.

We calculate the correlation between the trial stores and each potential control store to measure the similarity of trends.


In [ ]:
def correlation(input_table, metric_col, trial_store_nbr):
    store_nbr = input_table['STORE_NBR'].unique()
    corr_table = []
    
    for store in store_nbr:
        # Align data for the trial store and the current store on the common time period (YEARMONTH)
        trial_store_data = input_table[input_table['STORE_NBR'] == trial_store_nbr][['YEARMONTH', metric_col]]
        store_data = input_table[input_table['STORE_NBR'] == store][['YEARMONTH', metric_col]]
        
        # Merge the data on the common YEARMONTH to ensure the same length
        merged_data = pd.merge(trial_store_data, store_data, on='YEARMONTH', suffixes=('_trial', '_control'))
        
        if len(merged_data) > 1:  # To avoid computing correlation with a single data point or no data
            # Calculate correlation
            corr_measure = np.corrcoef(merged_data[f'{metric_col}_trial'], merged_data[f'{metric_col}_control'])[0, 1]
            corr_table.append({'Store1': trial_store_nbr, 'Store2': store, 'corr_measure': corr_measure})
        else:
            # If no data is available for comparison, append NaN
            corr_table.append({'Store1': trial_store_nbr, 'Store2': store, 'corr_measure': np.nan})
    
    return pd.DataFrame(corr_table)


In [ ]:
trial_store=77
corr_nSales=correlation(pre_trial_period,'TOTAL_SALES',trial_store)
corr_nSales.sort_values(by='corr_measure',ascending=False)

Correlation measures trend similarity, but stores can have different absolute sales numbers. 
Magnitude distance calculates the difference in absolute values between the trial and control stores. 
The distances are standardized to a 0-1 scale.


In [ ]:
def calculate_magnitude_distance(input_table, metric_col, store_comparison):
    store_numbers = input_table['STORE_NBR'].unique()
    dist_table = []
    
    for store in store_numbers:
        # Align data for the trial store and the current store based on common YEARMONTH
        store_comparison_data = input_table[input_table['STORE_NBR'] == store_comparison][['YEARMONTH', metric_col]]
        store_data = input_table[input_table['STORE_NBR'] == store][['YEARMONTH', metric_col]]
        
        # Merge the data to ensure both have the same time period
        merged_data = pd.merge(store_comparison_data, store_data, on='YEARMONTH', suffixes=('_comparison', '_store'))
        
        if len(merged_data) > 1:  # Ensure enough data points to calculate the magnitude distance
            # Calculate the mean absolute difference between the stores' metrics
            measure = abs(merged_data[f'{metric_col}_comparison'].values - merged_data[f'{metric_col}_store'].values).mean()
            dist_table.append({'Store1': store_comparison, 'Store2': store, 'measure': measure})
        else:
            # If no data is available for comparison, append NaN
            dist_table.append({'Store1': store_comparison, 'Store2': store, 'measure': np.nan})
    
    # Convert the list to a DataFrame
    dist_df = pd.DataFrame(dist_table)

    # Standardize the magnitude distances (0 to 1 scale)
    min_max_dist = dist_df.groupby(['Store1']).agg(minDist=('measure', 'min'), maxDist=('measure', 'max')).reset_index()
    dist_table = dist_df.merge(min_max_dist, on='Store1')
    
    # Calculate the standardized magnitude distance (0 to 1 scale)
    dist_table['magnitudeMeasure'] = 1 - (dist_table['measure'] - dist_table['minDist']) / (dist_table['maxDist'] - dist_table['minDist'])
    
    # Return the final table with averaged magnitude measures for each store comparison
    return dist_table.groupby(['Store1', 'Store2']).agg(mag_measure=('magnitudeMeasure', 'mean')).reset_index()



In [ ]:
magnitude_nSales = calculate_magnitude_distance(pre_trial_period, 'TOTAL_SALES', 77)
magnitude_nSales.sort_values(by='mag_measure',ascending=False)

In [ ]:
#combining correlation and magintude distance for total sales 
corr_weight=0.5
corr_and_mag_sales=corr_nSales.merge(magnitude_nSales,on=['Store1','Store2'])
corr_and_mag_sales['corr_mag_sales_score']=corr_and_mag_sales['corr_measure'] * corr_weight + corr_and_mag_sales['mag_measure'] * (1-corr_weight)

In [ ]:
corr_and_mag_sales.sort_values(by='corr_mag_sales_score',ascending=False)

In [ ]:
# For number of customers
corr_customer=correlation(pre_trial_period,'TOTAL_CUSTOMER',77)
mag_customer=calculate_magnitude_distance(pre_trial_period,'TOTAL_CUSTOMER',77)

#combining correlation and magnitude distance for total number of customers 

corr_and_mag_cust=corr_customer.merge(mag_customer,on=['Store1','Store2'])
corr_and_mag_cust['corr_mag_cust_score']=corr_and_mag_cust['corr_measure']*0.5+corr_and_mag_cust['mag_measure']*(1-0.5)

In [ ]:
#combining correlation and magnitude distance of total sales and total number of customers along with their scores on store1 and store 2
final_score=corr_and_mag_sales.merge(corr_and_mag_cust,on=['Store1','Store2'])
final_score['final_score']=final_score['corr_mag_sales_score']*0.5+final_score['corr_mag_cust_score']*(1-0.5)


In [ ]:
final_score.sort_values(by='final_score',ascending=False)

In [ ]:
control_store = final_score.loc[final_score['Store1'] == trial_store].sort_values('final_score', ascending=False).iloc[1]['Store2']
control_store

VISUALIZATION FOR TOTAL SALES OVER MONTH 

In [ ]:
control_store_nbr=233
trial_store_nbr=77
control_store_sales=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_SALES']]
trial_store_sales=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_SALES']]
other_store_sales=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_SALES']]

control_store_sales['YEARMONTH'] = pd.to_datetime(control_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
trial_store_sales['YEARMONTH'] = pd.to_datetime(trial_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
other_store_sales['YEARMONTH'] = pd.to_datetime(other_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')


In [ ]:

plt.figure(figsize=(10, 6))

sns.lineplot(data=control_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='control_store')
sns.lineplot(data=trial_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='trial_store')
sns.lineplot(data=other_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='other_store')
plt.title('TOTAL SALES BY MONTH OF TRIAL STORE 77 AND CONTROL STORE 233 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL SALES')
plt.legend()
plt.show()

VISUALIZATION FOR TOTAL CUSTOMERS BY MONTHS 

In [ ]:
control_store_nbr=233
trial_store_nbr=77
control_store_cust=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
trial_store_cust=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
other_store_cust=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_CUSTOMER']]

control_store_cust['YEARMONTH'] = pd.to_datetime(control_store_cust['YEARMONTH'].astype(str)+'01', format='%Y%m%d')
trial_store_cust['YEARMONTH'] = pd.to_datetime(trial_store_cust['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
other_store_cust['YEARMONTH'] = pd.to_datetime(other_store_cust['YEARMONTH'].astype(str) + '01', format='%Y%m%d')


In [ ]:
control_store_cust

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=control_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='control_store')
sns.lineplot(data=trial_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='trial_store')
sns.lineplot(data=other_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='other_store')
plt.title('TOTAL CUSTOMERS BY MONTH OF TRIAL STORE 77 AND CONTROL STORE 233 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL CUSTOMER')
plt.legend()
plt.show()


                                                        FOR TRIAL STORE 86


FOR TOTAL SALES

In [ ]:
corr_sales=correlation(pre_trial_period,'TOTAL_SALES',86)
dist_sales=calculate_magnitude_distance(pre_trial_period,'TOTAL_SALES',86)

In [ ]:
#combining correaltion and distance metric of trial store number 86
corr_weight=0.5
corr_dist_sales=corr_sales.merge(dist_sales,on=['Store1','Store2'])
corr_dist_sales['corr_mag_sales_score']=corr_dist_sales['corr_measure']*corr_weight+corr_dist_sales['mag_measure']*corr_weight

In [ ]:
corr_dist_sales.sort_values(by='corr_mag_sales_score',ascending=False)

FOR TOTAL CUSTOMERS

In [ ]:
corr_cust=correlation(pre_trial_period,'TOTAL_CUSTOMER',86)
dist_cust=calculate_magnitude_distance(pre_trial_period,'TOTAL_CUSTOMER',86)

In [ ]:
#combining correaltion and distance metric of trial store number 86
corr_weight=0.5
corr_dist_cust=corr_cust.merge(dist_cust,on=['Store1','Store2'])
corr_dist_cust['corr_mag_cust_score']=corr_dist_cust['corr_measure']*corr_weight+corr_dist_cust['mag_measure']*corr_weight

In [ ]:
corr_dist_cust.sort_values(by='corr_mag_cust_score',ascending=False)


In [ ]:
#combining correlation and magnitude distance of total sales and total number of customers along with their scores on store1 and store 2 for trial store 86
final_score=corr_dist_sales.merge(corr_dist_cust,on=['Store1','Store2'])
final_score['final_score']=final_score['corr_mag_sales_score']*0.5+final_score['corr_mag_cust_score']*(1-0.5)


In [ ]:
final_score.sort_values(by='final_score',ascending=False)


In [ ]:
new_trial_store=86
control_store_for_trial_store_86=final_score.loc[final_score['Store1']==new_trial_store].sort_values(by='final_score',ascending=False).iloc[1]['Store2']

In [ ]:
control_store_for_trial_store_86

LET'S CONFIRM OUR ANALYSIS USING VISUALIZATION 

VISUALIZATION FOR TOTAL SALES OVER MONTH 

In [ ]:
control_store_nbr=155
trial_store_nbr=86
control_store_sales=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_SALES']]
trial_store_sales=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_SALES']]
other_store_sales=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_SALES']]


control_store_sales['YEARMONTH'] = pd.to_datetime(control_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
trial_store_sales['YEARMONTH'] = pd.to_datetime(trial_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
other_store_sales['YEARMONTH'] = pd.to_datetime(other_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=control_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='control_store')
sns.lineplot(data=trial_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='trial_store')
sns.lineplot(data=other_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='other_store')
plt.title('TOTAL SALES BY MONTH OF TRIAL STORE 86 AND CONTROL STORE 155 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL SALES')
plt.legend()
plt.tight_layout()
plt.show()

VISUALISATION FOR TOTAL CUSTOMERS FOR TRIAL STORE 86 AND CONTROL STORE 155 ALONG WITH OTHER STORE CUSTOMERS.


In [ ]:
control_store_nbr=155
trial_store_nbr=86
control_store_cust=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
trial_store_cust=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
other_store_cust=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_CUSTOMER']]

control_store_cust['YEARMONTH']=pd.to_datetime(control_store_cust['YEARMONTH'].astype('str'),format='%Y%m')
trial_store_cust['YEARMONTH']=pd.to_datetime(trial_store_cust['YEARMONTH'].astype('str'),format='%Y%m')
other_store_cust['YEARMONTH']=pd.to_datetime(other_store_cust['YEARMONTH'].astype('str'),format='%Y%m')


In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=control_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='control store')
sns.lineplot(data=trial_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='trial store')
sns.lineplot(data=other_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='other store')
plt.title('TOTAL CUSTOMERS BY MONTH OF TRIAL STORE 86 AND CONTROL STORE 155 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL SALES')
plt.legend()
plt.tight_layout()
plt.show()


                                                        FOR TRIAL STORE 88


FOR TOTAL SALES

In [ ]:
corr_sales=correlation(pre_trial_period,'TOTAL_SALES',88)
dist_sales=calculate_magnitude_distance(pre_trial_period,'TOTAL_SALES',88)

In [ ]:
#combining correaltion and distance metric of trial store number 86
corr_weight=0.5
corr_dist_sales=corr_sales.merge(dist_sales,on=['Store1','Store2'])
corr_dist_sales['corr_mag_sales_score']=corr_dist_sales['corr_measure']*corr_weight+corr_dist_sales['mag_measure']*corr_weight

In [ ]:
corr_dist_sales.sort_values(by='corr_mag_sales_score',ascending=False)


FOR TOTAL CUSTOMER


In [ ]:
corr_cust=correlation(pre_trial_period,'TOTAL_CUSTOMER',88)
dist_cust=calculate_magnitude_distance(pre_trial_period,'TOTAL_CUSTOMER',88)

In [ ]:
#combining correaltion and distance metric of trial store number 88
corr_weight=0.5
corr_dist_cust=corr_cust.merge(dist_cust,on=['Store1','Store2'])
corr_dist_cust['corr_mag_cust_score']=corr_dist_cust['corr_measure']*corr_weight+corr_dist_cust['mag_measure']*corr_weight

In [ ]:
corr_dist_cust.sort_values(by='corr_mag_cust_score',ascending=False)


In [ ]:
#combining correlation and magnitude distance of total sales and total number of customers along with their scores on store1 and store 2 for trial store 88
final_score=corr_dist_sales.merge(corr_dist_cust,on=['Store1','Store2'])
final_score['final_score']=final_score['corr_mag_sales_score']*0.5+final_score['corr_mag_cust_score']*(1-0.5)


In [ ]:
final_score.sort_values(by='final_score',ascending=False)


In [ ]:
trial_store=88
control_store_for_trial_store_88=final_score.loc[final_score['Store1']==trial_store].sort_values(by='final_score',ascending=False).iloc[2]['Store2']

In [ ]:
control_store_for_trial_store_88

I KNOW YOU WONDER ABOVE WHY I SELECT 237 AS CONTROL STORE FOR TRIAL STORE 88 INSTEAD OF STORE NUMBER 178 WHERE IT RANKED AS SECOND HERE IS WHY I SELECT 237 AS CONTROL STORE BECAUSE WHEN YOU CAREFULLY NOTICE THAT THE FINAL SCORE OF STORE NUMBER 237 AND 178 IS APPROX 0.80 BUT THE DIFFERENCE WHICH MAKE 237 AS CONTROL STORE IS THAT CORRELATION AND MAGNITUDE SCORE FOR TOTAL CUSTOMER IS MORE FOR 237 THAN 178 AND ALONE MAGNITUDE MEASURE FOR TOTAL SALES AND TOTAL CUSTOMER IS HEIGHER THAN STORE NUMBER 178 WHICH LEAD ME TO CHOICE IT AS CONTROL STORE 

BTW BOTH STORE ARE GIVING TOUGH FIGHT. YOU CAN VERFIY IT BY VISUAL IT 



LET'S CONFIRM OUR ANALYSIS USING VISUALIZATION

VISUALISATION FOR TOTAL SALES OVER MONTH

In [ ]:
control_store_nbr=237
trial_store_nbr=88
control_store_sales=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_SALES']]
trial_store_sales=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_SALES']]
other_store_sales=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_SALES']]


control_store_sales['YEARMONTH'] = pd.to_datetime(control_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
trial_store_sales['YEARMONTH'] = pd.to_datetime(trial_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')
other_store_sales['YEARMONTH'] = pd.to_datetime(other_store_sales['YEARMONTH'].astype(str) + '01', format='%Y%m%d')

plt.figure(figsize=(10, 6))
sns.lineplot(data=control_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='control_store')
sns.lineplot(data=trial_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='trial_store')
sns.lineplot(data=other_store_sales,x='YEARMONTH',y='TOTAL_SALES',label='other_store')
plt.title('TOTAL SALES BY MONTH OF TRIAL STORE 88 AND CONTROL STORE 237 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL SALES')
plt.legend()
plt.tight_layout()
plt.show()

VISUALISATION FOR TOTAL CUSTOMERS FOR TRIAL STORE 86 AND CONTROL STORE 155 ALONG WITH OTHER STORE CUSTOMERS.

In [ ]:
control_store_nbr=237
trial_store_nbr=88
control_store_cust=metrics[metrics['STORE_NBR']==control_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
trial_store_cust=metrics[metrics['STORE_NBR']==trial_store_nbr][['YEARMONTH','TOTAL_CUSTOMER']]
other_store_cust=metrics[~metrics['STORE_NBR'].isin([control_store_nbr,trial_store_nbr])][['YEARMONTH','TOTAL_CUSTOMER']]

control_store_cust['YEARMONTH']=pd.to_datetime(control_store_cust['YEARMONTH'].astype('str'),format='%Y%m')
trial_store_cust['YEARMONTH']=pd.to_datetime(trial_store_cust['YEARMONTH'].astype('str'),format='%Y%m')
other_store_cust['YEARMONTH']=pd.to_datetime(other_store_cust['YEARMONTH'].astype('str'),format='%Y%m')

plt.figure(figsize=(10, 6))
sns.lineplot(data=control_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='control store')
sns.lineplot(data=trial_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='trial store')
sns.lineplot(data=other_store_cust,x='YEARMONTH',y='TOTAL_CUSTOMER',label='other store')
plt.title('TOTAL CUSTOMERS BY MONTH OF TRIAL STORE 88 AND CONTROL STORE 237 ALONG WITH OTHER STORE',fontsize=8)
plt.ylabel('TOTAL SALES')
plt.legend()
plt.tight_layout()
plt.show()

CONCLUSION:
WE FOUND  STORE 233,155,237 AS CONTROL STORE FOR TRIAL STORE 77,86,88